In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import librareis & data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import VotingRegressor

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/test.csv')
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv')
train.shape, test.shape

# Data Exploration

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
sns.heatmap(train.isnull(), cbar = False);

# Data Visualization

In [ ]:
train.hist(bins = 30, figsize = (20,20), color = 'b')

# Correlation Analysis

In [ ]:
correlations = train.corr()
f, ax = plt.subplots(figsize = (20,20))
sns.heatmap(correlations, annot = True)

In [ ]:
correlations = train.corr()['target'].sort_values()
correlations

In [ ]:
X = train.drop(['id','target'], axis = 1)
y = train['target']
test = test.drop(['id'], axis= 1)

# Paramenters XGBoost, LGBM, CatBoost

In [ ]:
#https://www.kaggle.com/hamzaghanmi/xgboost-hyperparameter-tuning-using-optuna

param_xgb = {'lambda': 0.0030282073258141168, 
             'alpha': 0.01563845128469084, 
             'colsample_bytree': 0.5,
             'subsample': 0.7,
             'n_estimators': 1000, 
             'learning_rate': 0.01,
             'max_depth': 15,
             'random_state': 2020, 
             'min_child_weight': 257,
             'tree_method':'gpu_hist',
             'predictor': 'gpu_predictor'}

# parameters from this awesome kernal 
# https://www.kaggle.com/hamditarek/tabular-playground-series-xgboost-lightgbm
param_lgbm ={'random_state': 33,'n_estimators':1000,
 'min_data_per_group': 5,
 'boosting_type': 'gbdt',
 'device_type' : 'gpu',
 'num_leaves': 256,
 'num_iterations' : 1000,
 'max_dept': -1,
 'learning_rate': 0.005,
 'subsample_for_bin': 200000,
 'lambda_l1': 1.074622455507616e-05,
 'lambda_l2': 2.0521330798729704e-06,
 'n_jobs': -1,
 'cat_smooth': 1.0,
 'silent': True,
 'importance_type': 'split',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'bagging_fraction': 0.8206341150202605,
 'min_data_in_leaf': 100,
 'min_sum_hessian_in_leaf': 0.001,
 'bagging_freq': 6,
 'feature_fraction': 0.5,
 'min_gain_to_split': 0.0,
 'min_child_samples': 20}

#https://www.kaggle.com/bhavikjain/lgbm-xgboost-catboost-tabular-playground-series
param_cat = {'iterations': 1000,
                             'learning_rate': 0.05,
                             'depth': 10,
                             'reg_lambda': 10,
                             'eval_metric': 'RMSE',
                             'bagging_temperature': 0.2,
                             'od_type': 'Iter',
                             'metric_period': 50,
                             'od_wait': 20
}

# Stacking Model

In [ ]:
%%time
kf = KFold(n_splits=5, shuffle=True, random_state=7)

oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(test.shape[0])

for n_fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]
    level10 = list()
    level10.append(('cbr', CatBoostRegressor(**param_cat, task_type='GPU')))
    level10.append(('lgr', LGBMRegressor(**param_lgbm)))
    level10.append(('xgb', XGBRegressor(**param_xgb)))
    level10.append(('mlp', MLPRegressor()))
    
    level1 = VotingRegressor(estimators = level10, n_jobs = -1)

    model = StackingRegressor(estimators=level10, final_estimator=level1, cv=5)
    model.fit(X_train, y_train)
    oof_preds[val_idx] = model.predict(X_valid)
    pred_xgb = model.predict(test)
    sub_preds += pred_xgb / kf.n_splits

print(np.sqrt(mean_squared_error(y, oof_preds)))

# Submission

In [ ]:
# Make submission
sample_submission['target'] = sub_preds
sample_submission.to_csv("stacking_submission.csv", index=False)
sample_submission.head()